# Testing the normal label formulation and the DL Enhancement / Lifted Formulation from https://link.springer.com/content/pdf/10.1007/s13676-012-0010-0.pdf

In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))
from imports import ilpgraph, readFile
import tsplib95
from imports import networkx as imp_nx
from network import min_tsp, gen_path_atsp, atsp_DT_formulation, tsp_callbacks
import networkx as nx
from gurobipy import *
import xlsxwriter
import random
import matplotlib.pyplot as plt



def __write_solution(solution:list):
    print("writing solutions...")
    workbook = xlsxwriter.Workbook('BenchmarksTSP.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('E1', 'Runtime', bold)
    worksheet.write('F1', 'Objective Value', bold)
    worksheet.write('G1', 'Constraints', bold)
    worksheet.write('H1', 'MIP Gap', bold)
    
    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    
        
def runBenchmark(paths:list):
    solutions = []
    i = 0
    
    for path in paths:
        tsp = tsplib95.load(path)
        
        G = tsp.get_graph()
        G = imp_nx.read(G)
        #m = gen_path_atsp.createGenModel(G,'min','metric')
        #m = atsp_DT_formulation.createGenModel(G, "min", "metric")
        m = tsp_callbacks.createGenModel(G, "min", "metric")
        m.params.MIPGap = 0.05
        m.params.TimeLimit = 1500
       

        m.optimize()

        solutions.append((i, len(G.G.edges()), len(G.G.nodes()), m.runTime, m.objVal, m.numConstrs, m.MIPGap))
        i += 1
    print(solutions)
    __write_solution(solutions)         

if __name__ == '__main__':
    

    paths = ["TSPTestInstances/p654.tsp"]
    
    runBenchmark(paths)


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-03-23
Using license file C:\Program Files\Gurobi\gurobi.lic
Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 4965 rows, 2739025 columns and 5479705 nonzeros
Model fingerprint: 0x1a850a53
Variable types: 0 continuous, 2739025 integer (2739025 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
In Callback
Presolve removed 1655 rows and 1655 columns (presolve time = 6s) ...
Presolve removed 1655 rows and 1655 columns
Presolve time: 7.80s
Presolved: 3310 rows, 2737370 columns, 5474740 nonzeros
Variable types: 0 continuous, 2737370 integ

GurobiError: Out of memory

# Heuristic implementation. Using Nearest Neighbour to create a new Tour and 2 Opt to improve the tour

In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))
from imports import ilpgraph, readFile
from imports import networkx as imp_nx
from network import tsp_heuristics as tsph
import networkx as nx
from gurobipy import *
import xlsxwriter
import random
import matplotlib.pyplot as plt


nbr_cities = 1655
path = r"TSPTestInstances/d1655.xml"
distance = readFile.read_tsplib(path)
for i in range(nbr_cities):
    distance[i][i] = 10000 # Changing the diagonal to "infinity"
        
tour, length = tsph.nearNeighHeur(distance, nbr_cities)
print(tour, length)
print("2 OPT")
tour, length = tsph.twoOptImprov(distance, tour, length, nbr_cities)
print(tour)
print(length)



[0, 12, 11, 17, 18, 24, 23, 29, 30, 36, 35, 41, 42, 48, 47, 53, 54, 60, 59, 65, 66, 72, 71, 74, 73, 82, 81, 84, 83, 90, 89, 95, 96, 94, 88, 85, 80, 75, 70, 64, 58, 52, 46, 40, 34, 28, 22, 16, 10, 9, 15, 21, 27, 33, 39, 45, 51, 57, 63, 69, 76, 79, 86, 87, 93, 91, 97, 100, 101, 107, 110, 113, 114, 117, 118, 121, 122, 125, 126, 127, 124, 123, 120, 119, 116, 115, 112, 111, 108, 102, 99, 98, 92, 77, 78, 67, 61, 55, 49, 43, 37, 31, 25, 19, 13, 8, 5, 4, 1, 2, 3, 6, 7, 14, 20, 26, 32, 38, 44, 50, 56, 62, 68, 620, 619, 618, 617, 616, 615, 614, 613, 612, 611, 610, 609, 608, 607, 606, 602, 595, 590, 580, 569, 562, 561, 560, 568, 581, 589, 594, 601, 600, 593, 582, 583, 584, 585, 586, 566, 558, 553, 551, 541, 530, 528, 521, 511, 501, 494, 500, 493, 510, 522, 529, 540, 552, 557, 565, 587, 588, 591, 598, 599, 592, 567, 559, 554, 550, 542, 531, 527, 520, 512, 502, 495, 499, 492, 473, 472, 471, 470, 469, 484, 485, 486, 487, 490, 496, 506, 516, 519, 526, 538, 543, 546, 555, 563, 573, 576, 577, 572, 571,

KeyboardInterrupt: 